### 導入套件

In [2]:
import yfinance as yf
import numpy as np
import pandas as pd
import random

### 常數設置

In [3]:
# 基因演算法參數
NUM_ASSETS = 20  # 股票數量
POPULATION_SIZE = 50  # 族群大小
GENERATIONS = 100  # 最大迭代次數
CROSSOVER_RATE = 0.8  # 交配機率
MUTATION_RATE = 0.01  # 突變機率

### 股價引入，並計算風險

In [4]:
# 隨機選擇台股代號
taiwan_stocks = [
    "2330.TW", "2317.TW", "2454.TW", "2308.TW", "6505.TW", 
    "2002.TW", "2882.TW", "2303.TW", "2881.TW", "1326.TW", 
    "3008.TW", "2412.TW", "3711.TW", "3045.TW", "2883.TW", 
    "2603.TW", "2891.TW", "2912.TW", "2610.TW", "2301.TW"
]

# 隨機選擇 NUM_ASSETS 支台股
selected_stocks = random.sample(taiwan_stocks, NUM_ASSETS)
print("Selected Stocks:", selected_stocks)

# 抓取歷史數據
def fetch_stock_data(stock_list, start_date="2020-01-01", end_date="2023-12-31"):
    data = {}
    for stock in stock_list:
        df = yf.download(stock, start=start_date, end=end_date)
        data[stock] = df['Adj Close']
    return pd.DataFrame(data)

# 獲取收盤價
stock_prices = fetch_stock_data(selected_stocks)

# 計算每日回報率
daily_returns = stock_prices.pct_change().dropna()

# 計算各股票的平均回報率與風險（標準差）
mean_returns = daily_returns.mean()
risks = daily_returns.std()

# 計算協方差矩陣
cov_matrix = daily_returns.cov()

Selected Stocks: ['2301.TW', '6505.TW', '2882.TW', '2881.TW', '3008.TW', '1326.TW', '2610.TW', '2002.TW', '2454.TW', '2883.TW', '2891.TW', '3711.TW', '2412.TW', '2330.TW', '2912.TW', '2317.TW', '2308.TW', '2303.TW', '3045.TW', '2603.TW']
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[**

### 基因演算法

In [5]:
# 適應函數（回報率與風險的權衡）
def fitness(chromosome):
    portfolio_return = np.dot(chromosome, mean_returns)
    portfolio_risk = np.sqrt(np.dot(chromosome.T, np.dot(cov_matrix, chromosome)))
    sharpe_ratio = portfolio_return / (portfolio_risk + 1e-6)  # 防止分母為0
    return sharpe_ratio

# 初始化族群
def initialize_population():
    return np.random.randint(0, 2, (POPULATION_SIZE, NUM_ASSETS))

# 選擇（輪盤選擇法）
def select(population, fitness_scores):
    total_fitness = sum(fitness_scores)
    probabilities = [score / total_fitness for score in fitness_scores]
    selected_indices = np.random.choice(range(POPULATION_SIZE), size=POPULATION_SIZE, p=probabilities)
    return population[selected_indices]

# 交配
def crossover(parent1, parent2):
    if random.random() < CROSSOVER_RATE:
        point = random.randint(1, NUM_ASSETS - 1)
        child1 = np.concatenate((parent1[:point], parent2[point:]))
        child2 = np.concatenate((parent2[:point], parent1[point:]))
        return child1, child2
    return parent1, parent2

# 突變
def mutate(chromosome):
    for i in range(NUM_ASSETS):
        if random.random() < MUTATION_RATE:
            chromosome[i] = 1 - chromosome[i]
    return chromosome

# 主函數
def genetic_algorithm():
    population = initialize_population()
    best_solution = None
    best_fitness = -float('inf')
    
    for generation in range(GENERATIONS):
        fitness_scores = np.array([fitness(chromosome) for chromosome in population])
        if max(fitness_scores) > best_fitness:
            best_fitness = max(fitness_scores)
            best_solution = population[np.argmax(fitness_scores)]
        
        selected_population = select(population, fitness_scores)
        next_generation = []
        
        for i in range(0, POPULATION_SIZE, 2):
            parent1, parent2 = selected_population[i], selected_population[i + 1]
            child1, child2 = crossover(parent1, parent2)
            next_generation.append(mutate(child1))
            next_generation.append(mutate(child2))
        
        population = np.array(next_generation)
        
        print(f"Generation {generation + 1}, Best Fitness: {best_fitness}")
    
    portfolio_return = np.dot(best_solution, mean_returns)
    portfolio_risk = np.sqrt(np.dot(best_solution.T, np.dot(cov_matrix, best_solution)))
    
    print("\nOptimal Solution:")
    print("Selected Assets:", [selected_stocks[i] for i in range(NUM_ASSETS) if best_solution[i] == 1])
    print("Portfolio Return:", portfolio_return)
    print("Portfolio Risk:", portfolio_risk)
    print("Sharpe Ratio:", best_fitness)

# 執行基因演算法
genetic_algorithm()

Generation 1, Best Fitness: 0.0902382676632563
Generation 2, Best Fitness: 0.0902382676632563
Generation 3, Best Fitness: 0.0902382676632563
Generation 4, Best Fitness: 0.0902382676632563
Generation 5, Best Fitness: 0.09101250099541328
Generation 6, Best Fitness: 0.09101250099541328
Generation 7, Best Fitness: 0.0945051986472587
Generation 8, Best Fitness: 0.09806025497714481
Generation 9, Best Fitness: 0.09806025497714481
Generation 10, Best Fitness: 0.10023330566050778
Generation 11, Best Fitness: 0.10023330566050778
Generation 12, Best Fitness: 0.10023330566050778
Generation 13, Best Fitness: 0.10023330566050778
Generation 14, Best Fitness: 0.10023330566050778
Generation 15, Best Fitness: 0.10023330566050778
Generation 16, Best Fitness: 0.10023330566050778
Generation 17, Best Fitness: 0.10044788181804162
Generation 18, Best Fitness: 0.10227804565399481
Generation 19, Best Fitness: 0.10413821831753488
Generation 20, Best Fitness: 0.10413821831753488
Generation 21, Best Fitness: 0.104